# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [8]:
# checking your current working directory
print(f"Current directory: {os.getcwd()}")

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(f"Event data file example: {file_path_list[0]}")

Current directory: /home/pelielo/projects/nd027-project2
Event data file example: /home/pelielo/projects/nd027-project2/event_data/2018-11-25-events.csv


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [9]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(f"Total number of rows in dataset: {len(full_data_rows_list)}")
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Total number of rows in dataset: 8056


In [11]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(f"Number of rows in new csv file: {sum(1 for line in f)}")

Number of rows in new csv file: 6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [17]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [18]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [19]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [29]:
# Columns: 
# 0 - artist
# 1 - firstName
# 2 - gender
# 3 - itemInSession
# 4 - lastName
# 5 - length
# 6 - level
# 7 - location
# 8 - sessionId
# 9 - song
# 10 - userId

## Table 1: song_plays_session

In [25]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## select artist, song_title, song_length from song_plays_session where session_id = 338 and item_in_session = 4
query = "CREATE TABLE IF NOT EXISTS song_plays_session "
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [40]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "insert into song_plays_session (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "values (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [48]:
query = "select artist, song_title, song_length from song_plays_session where session_id = 338 and item_in_session = 4"
# query = "select count(*) from song_plays_session"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(count=6820)


## Table 2: song_plays_user

In [49]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## select artist, song_title, user_first_name, user_last_name from song_plays_user where user_id = 10 and session_id = 182
query = "CREATE TABLE IF NOT EXISTS song_plays_user "
query = query + "(user_id int, session_id int, song_title text, artist text, user_first_name text, user_last_name text, item_in_session int, PRIMARY KEY (user_id, session_id, song_title, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [51]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "insert into song_plays_user (user_id, session_id, song_title, artist, user_first_name, user_last_name, item_in_session)"
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), line[9], line[0], line[1], line[4], int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [53]:
query = "select artist, song_title, user_first_name, user_last_name from song_plays_user where user_id = 10 and session_id = 182"
# query = "select count(*) from song_plays_user"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist='Three Drives', song_title='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist='Down To The Bone', song_title="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')


## Table 3: song_plays_song

In [54]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## select user_first_name, user_last_name from song_plays_song where song_title = 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS song_plays_song "
query = query + "(song_title text, user_first_name text, user_last_name text, PRIMARY KEY (song_title, user_first_name, user_last_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [55]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "insert into song_plays_song (song_title, user_first_name, user_last_name)"
        query = query + "values (%s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [63]:
query = "select user_first_name, user_last_name from song_plays_song where song_title = 'All Hands Against His Own'"
# query = "select count(*) from song_plays_song"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(user_first_name='Jacqueline', user_last_name='Lynch')
Row(user_first_name='Sara', user_last_name='Johnson')
Row(user_first_name='Tegan', user_last_name='Levine')


### Drop the tables before closing out the sessions

In [64]:
## Drop the tables before closing out the session

query = "drop table if exists song_plays_session"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists song_plays_user"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "drop table if exists song_plays_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [65]:
session.shutdown()
cluster.shutdown()